In [171]:
import ftplib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce
import paramiko 
import datetime
from datetime import date
import re

In [172]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)
paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')

In [173]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy)==1:
    mes_hoy = "0"+mes_hoy
if len(dia_hoy)==1:
    dia_hoy = "0"+dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost =  "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost   

#Direccion descarga local
direccionDescargaLocal = "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost 
    
print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2023-09-01
01092023
VentaCARE_ASSISTANCE_01092023.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_01092023.xlsx


In [174]:
host="sftp.bciseguros.cl"
puerto=22
transporte = paramiko.Transport((host,puerto))
usuario= "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None,usuario,clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo

In [175]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath,localpath)


In [176]:
if sftp: sftp.close()
if transporte: transporte.close()

Inicio Manipulacion de Excel descargado

In [253]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(2382, 17)

In [254]:
archivoBCI.dtypes

POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [255]:
archivoBCI.RUT_ASEGURADO.fillna("0", inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
#archivoBCI["CELULAR"] = archivoBCI["CELULAR"].str.replace(" 0 ", "")

archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,25353688,8,MASCULINO,HIJO,MAXIMILIANO ALONSO LPEZ ARANCIBIA,2016-04-16,TIARE.ARANCIBIA.VERGARA@GMAIL.,NO INFORMADO,0
1,2611247,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17589680,5,17589680,5,FEMENINO,TITULAR MASCULINO,YENIFER VALERIA ROJAS ROMAN,1990-06-19,yenifer.rojas.r@gmail.com,SANTIAGO,954068927
2,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,17308982,1,FEMENINO,TITULAR MASCULINO,VALESKA BELEN SILVA BUSTOS,1989-08-04,kekita.bonita@hotmail.com,SANTIAGO,953810699
3,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,27712391,6,MASCULINO,HIJO,SANTIAGO JESS SALAZAR SILVA,2022-02-02,NO INFORMADO,NO INFORMADO,0
4,2611259,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,9711692,K,9711692,K,FEMENINO,TITULAR MASCULINO,ANA MARITZA CORTES LIZARDI,1964-08-04,ARIVERA.Y@ICLOUD.COM,FRANCISCO ALDAY 3457 COMPAÑIA ALTA- CIUDAD DEL...,962801697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,2643881,2023-08-30,2024-08-29,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,11256204,4,11256204,4,FEMENINO,TITULAR MASCULINO,IBARRA GALLARDO SYLVIA JACQUELINE,1968-06-21,,SANTIAGO,997639739
2328,2643883,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,11418540,K,11418540,K,FEMENINO,TITULAR FEMENINO,PATRICIA INES BAEZA URRA,1969-06-16,,SANTIAGO,977751485
2329,2643884,2023-08-30,2024-08-29,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16715039,K,16715039,K,FEMENINO,TITULAR FEMENINO,JASMIN ANDREA ARIAS LEIVA,1987-08-14,yo_ch_y20@hotmail.com,CALLE BOLIVIA 2501,991815552
2330,2643875,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,19515712,K,19515712,K,MASCULINO,TITULAR MASCULINO,DEMIAN NICOLAS GUTIERREZ ACUÑA,1996-09-28,SANDRA.ACUNA2@GMAIL.COM,NO INFORMADO,0


In [256]:
def isValid(email):
    if re.fullmatch(regex, email):
      return email
    else:
      return ""
    
def nombreFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  """ apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)>=2: """
  nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
  return nombresJuntos
  """ else:     
    return apellidos """
  print (len(apellidos))
  
def apellidoFuncion(nombreCompleto):
  """ nombreCompleto = str(nombreCompleto)
  apellidos = re.split(',', nombreCompleto)
  if len(apellidos)>=4: """
  nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
  return nombresJuntos
  """ else:     
    return apellidos """
  print (len(apellidos))  
  
""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """
    
  #archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
  #archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
  #print (apellidos)

' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [257]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {"no informado": "", "SN": ""}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")

""" archivoBCI["nombre"] = archivoBCI["NOMBRE_ASEGURADO"].apply(lambda y: nombreFuncion(y)).str.title()
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].apply(lambda y: apellidoFuncion(y)).str.title() """

archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2).str.title()
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2).str.title()
archivoBCI["nombres"] = archivoBCI["nombres"].apply(lambda y: nombreFuncion(y))
#archivoBCI["apellidos"] = archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y))


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13816\555546750.py:60: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [258]:
archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,DIREC,CELULAR,Rut Titular,Rut Carga,sexo,email,direccion,celular,nombres,apellidos
0,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,25353688,8,...,NO INFORMADO,,166479770,253536888,Hombre,,,,n a,NaN
1,2611247,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17589680,5,17589680,5,...,SANTIAGO,954068927,175896805,175896805,Mujer,yenifer.rojas.r@gmail.com,Santiago,954068927,n a,NaN
2,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,17308982,1,...,SANTIAGO,953810699,173089821,173089821,Mujer,kekita.bonita@hotmail.com,Santiago,953810699,n a,NaN
3,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,27712391,6,...,NO INFORMADO,,173089821,277123916,Hombre,,,,n a,NaN
4,2611259,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,9711692,K,9711692,K,...,FRANCISCO ALDAY 3457 COMPAÑIA ALTA- CIUDAD DEL...,962801697,9711692K,9711692K,Mujer,arivera.y@icloud.com,Francisco Alday 3457 Compañia Alta- Ciudad Del...,962801697,n a,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,2643881,2023-08-30,2024-08-29,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,11256204,4,11256204,4,...,SANTIAGO,997639739,112562044,112562044,Mujer,,Santiago,997639739,n a,NaN
2328,2643883,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,11418540,K,11418540,K,...,SANTIAGO,977751485,11418540K,11418540K,Mujer,,Santiago,977751485,n a,NaN
2329,2643884,2023-08-30,2024-08-29,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16715039,K,16715039,K,...,CALLE BOLIVIA 2501,991815552,16715039K,16715039K,Mujer,yo_ch_y20@hotmail.com,Calle Bolivia 2501,991815552,n a,NaN
2330,2643875,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,19515712,K,19515712,K,...,NO INFORMADO,,19515712K,19515712K,Hombre,sandra.acuna2@gmail.com,,,n a,NaN


In [259]:
#archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
